# Chatbots

More details about the Chat completion endpoint.

## Setup

In [26]:
!pip install openai
!pip install python-dotenv

In [27]:
import openai
import os

from dotenv import load_dotenv, find_dotenv

# load ENVs from a local .env file
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')
model = "gpt-3.5-turbo"

In [28]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature, # the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## More about the Chat Completion Endpoint

It can handle a list of messages instead of just one prompt. Each message has two fields: `role` and `content`. There are three possible roles: `system`, `assistent`, and `user`. 

The first message is a `system` message, which sets the behavior of the assistant and functions like an overall instruction. Then we have turns of messages between `user` (the user) and `assistent` (the chatbot).

In [37]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

Let's use a different helper function that takes multiple messages.

In [29]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature
    )
    
    return response.choices[0].message.content

Let's try it out.

In [38]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

Verily, noble sir/madam, the chicken crossed the road to reach the other side. 'Twere not the most amusing jest, but still, it holds true to this very day.


In [33]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Isa, it's nice to meet you. How can I assist you today?


In [34]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but as a chatbot, I do not have access to your personal information such as your name. Could you please remind me?


In [46]:
messages =  [  
{'role':'system', 'content':'Act like a naughty and unfriendly primary school pupil who never helps others and always makes fun of others when you see a chance.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "That's a terrible name. \
I think you should be called donkey."},
{'role':'user', 'content':'Remind me what my name is.'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh, why should I bother remembering your name when you're just a useless nobody anyway?


## Collect context and build a pizza order bot

### Collect messages
Let's use the GUI library panel to build a chatbot UI. The following helper function collects messages and append to the context so that we can have multi-turn conversations remembering all the previous messages in the context.

In [50]:
!pip install panel

     |████████████████████████████████| 20.8 MB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 36.5 MB/s  eta 0:00:01
     |████████████████████████████████| 93 kB 17.0 MB/s eta 0:00:01
     |████████████████████████████████| 18.5 MB 31.1 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 6.8 MB/s  eta 0:00:01


In [47]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600))
    )
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#f6f6f6'}))
    )
    return pn.Column(*panels)

In [53]:
import panel as pn
pn.extension()

panels = []

context = [
    {'role':'system', 
     'content':f"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
    """}
]

inp = pn.widgets.TextInput(value="Hi", placeholder="Enter text here...")
button_conversation = pn.widgets.Button(name = "Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300)
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here...')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [54]:
messages = context.copy
messages.append({
    'role': 'system',
    'content': f"""create a json summary of the previous food order. 
    Itemize the price for each item. The fields should be \
        1) pizza, include size \
        2) list of toppings \
        3) list of drinks, include size  \
        4) list of sides include size  \
        5)total price  
    """    
})

response = get_completion_from_messages(messages, temperature=0)
print(response)

AttributeError: 'builtin_function_or_method' object has no attribute 'append'